In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import *
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob
from tweets_ops import *
from url_helpers import analyze_url
from matching import *

In [2]:
#DailyMail -> Bad
#NYTimes -> Neutral for acsh Top for climatefeedback
#the Atlantic -> Top for acsh Neutral for climatefeedback

In [3]:
df = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

In [4]:
df[df.apply(lambda x: analyze_url(x['url'])[0]=='dailymail.co.uk', axis=1)].to_csv(cache_dir+'dailymail.tsv', sep='\t', index=None)

aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v2.tsv', cache_dir+'dailymail.tsv', cache_dir+'dailymail.tsv')


In [5]:
df = pd.read_csv(cache_dir+'dailymail.tsv', sep='\t').fillna(0)

In [6]:
case = 'dailymail'
for s in ['full', 'paragraph', 'sentence']:
    train = pd.read_csv(cache_dir+'similarity_model/train_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    train = train[train.related==True]
    test = pd.read_csv(cache_dir+'similarity_model/test_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    pd.concat([train, test]).to_csv(cache_dir+'similarity_model/'+case+'/'+s+'.tsv', sep='\t', index=None)
    
test_similarity_model(cache_dir+'similarity_model/'+case+'/', cache_dir+'similarity_model/rf_model.sav', cache_dir+'similarity_model/'+case+'/results.tsv')
sim = pd.read_csv(cache_dir+'similarity_model/'+case+'/results.tsv', sep='\t').drop('paper', axis=1).groupby('article').max().reset_index()
df = df.merge(sim, left_on='url', right_on='article').drop('article', axis=1)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.0. This might lead to breakin

In [12]:
#df[['url', 'title', 'authors', 'quote_indicators', 'likes', 'replies',
#        'replies_count', 'replies_mean_polarity', 'replies_mean_subjectivity',
#        'retweets', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
#        'tweets_time_delta', 'users_countries', 'users_median_followers',
#        'users_median_friends', 'users_median_tweets', 'related']]

from textstat.textstat import textstat

In [14]:
df['full_text'].apply(lambda x: textstat.flesch_reading_ease(x))

0    61.87
1    54.46
2    67.38
3    59.23
Name: full_text, dtype: float64

In [8]:
sim

,article,related
0,http://www.dailymail.co.uk/health/article-4444...,0.100000
1,http://www.dailymail.co.uk/health/article-4953...,0.986250
2,http://www.dailymail.co.uk/health/article-5109...,0.066667
3,http://www.dailymail.co.uk/news/article-528470...,0.936250
